## Video resources, configuration

Define a **VIDEO_RESOURCES** environment variable pointing to the location of your original videos, and **VIDEO_COMPRESSED** environment variable pointing to the location to use for storing the resulting compressed videos.

In [6]:
from dotenv import dotenv_values
import os

config = dict()
config["VIDEO_RESOURCES"] = os.getenv("VIDEO_RESOURCES", None)
config["VIDEO_COMPRESSED"] = os.getenv("VIDEO_COMPRESSED", None)

if not any(config.values()):
    config = dotenv_values(os.path.join(os.path.dirname(os.getcwd()), ".env"))

In [12]:
# input sources
config["VIDEO_IN"] = [os.path.join(config["VIDEO_RESOURCES"], x) \
                      for x in os.listdir(config["VIDEO_RESOURCES"]) if \
                      x.endswith(".mp4") or x.endswith(".mkv")]

In [13]:
# destination
config["VIDEO_OUT"] = [x.replace("/original/", "/compressed/") for x in config["VIDEO_IN"]]

In [14]:
import ffmpeg

from ipywidgets import interact
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [70]:
probe = ffmpeg.probe(config["VIDEO_IN"][0])
video_info = next(stream for stream in probe["streams"] if stream["codec_type"] == "video")
# display the metadata within the sequence
#print(video_info)

In [71]:
width = int(video_info["width"])
height = int(video_info["height"])

num_frames = 0

if "nb_frames" in video_info:
    num_frames = int(video_info["nb_frames"])
else:
    hours, mins, secs = video_info["tags"]["DURATION"].split(":")
    secs,frames = secs.split(",")
    hours, mins, secs, frames = map(lambda x: int(x) if len(x) < 3 else \
                                    round(int(x)*1e-7), [hours, mins, secs, frames])
    frame_rate = int(video_info["r_frame_rate"].split("/")[0])
    num_frames = frame_rate * (3600 * hours + 60 * mins + secs) + frames
    
#print("total number of frames in sequence = {}".format(num_frames))

In [72]:
out, err = (
    ffmpeg
    .input(config["VIDEO_IN"][0])
    .output("pipe:", format="rawvideo", pix_fmt="rgb24")
    .run(capture_stdout = True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /

In [74]:
@interact(frame=(0, num_frames))
def show_frame(frame = 0):
    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=274), Output()), _dom_classes=('widget-inter…

In [75]:
#print(video_info)

Check codecs with ```ffmpeg -codecs``` and specific options with ```ffmpeg -h encoder=<encoder here>```, example for the h264 case, ```ffmpeg -h encoder=libx264```.
In our case, we're interested in h264, h264/HEVC, h266/VVC, VP8, VP9, AV1.

In [76]:
!ffmpeg -codecs|grep -wE 'av1|vp8|vp9|h264|hevc|vvc'

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /

## Codec options

### H.264 | MPEG-4 AVC

We need to check the options for each family of codecs, though to keep it manageable, we'll be restricted to changing the macroblock size, and the number of intraframes, as well as perhaps the pixel format for specific luma and chroma subsampling. We might change the DCT, IDCT type, from fast integer to floating point, and do some analysis first of common metrics, then of new perceptual metrics, then using deep learning and reference free image quality metrics.

In [77]:
!ffmpeg -h encoder=libx264

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /

#### Encoding Entropy Algorithm *(coder)* can be one of:
 * [CABAC](https://en.wikipedia.org/wiki/Context-adaptive_binary_arithmetic_coding) - Context Adaptive Binary Arithmetic Coding
 * [CAVLC](https://en.wikipedia.org/wiki/Context-adaptive_variable-length_coding) - Context Adaptive Variable Length Encoding
 * [AC](https://en.wikipedia.org/wiki/Arithmetic_coding) - Arithmetic Coding
 * [VLC](https://en.wikipedia.org/wiki/Variable-length_code) - Variable Lenght Encoding

See [also this](https://learn.akamai.com/en-us/webhelp/media-services-on-demand/media-services-on-demand-encoder-best-practices/GUID-F3F0BD92-69C3-4B7F-BE84-A33BA138B55A.html)

Rate control, can be one of:
 * ConstQP - Constant Quantization P mode
 * [VBR](https://en.wikipedia.org/wiki/Variable_bitrate) - Variable Bit Rate
 * [CBR](https://en.wikipedia.org/wiki/Constant_bitrate) - Constant Bit Rate
 * CBR_LD_HQ - Constant Bit Rate with Low Delay, High Quality mode
 * CBR_HQ - Constant Bit Rate High Quality Mode
 * VBR_HQ - Variable Bit Rate High Quality Mode

[Chroma subsampling](https://en.wikipedia.org/wiki/Chroma_subsampling) can be YUV 4:2:0, 4:2:2, 4:4:4.
The libx264 codec has more control over the macroblocks when compared with NVenc. Other codecs support other sampling modes, such as YUV 4:1:1, and quantization modes for 10bit, 12bit, 16bit signals, besides RGB.

### h265 | HEVC options

In [66]:
# hevc, libx265, kvazaar, hevc_nvenc
!ffmpeg -h encoder=libx265

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /

The same criteria as with h264 apply.

### H.266 | VVC

It's known to FFMPEG, but without a supported encoder, yet, though patches exists in development branches.
Test with the official Fraunhoffer provided VVenc.
It seems to expect a YUV input video, already chroma subsampled in 4:2:0, 8, or 10bits

In [68]:
#!ffmpeg -h encoder=vvc
!vvencapp --help


vvencapp: Fraunhofer VVC Encoder ver. 1.3.0 [Linux][GCC 10.3.0][64 bit][SIMD=AVX2]

#======== General Options ================
        --help [0]              show default help
        --fullhelp [0]          show full help
  -v,   --verbosity [verbose]   Specifies the level of the verboseness (0: silent, 1: error, 2: warning, 3: info, 4:
                                notice, 5: verbose, 6: debug)
        --version [0]           show version 

#======== Input Options ================
  -i,   --input []              original YUV input file name or '-' for reading from stdin
  -s,   --size [1920x1080]      specify input resolution (WidthxHeight)
  -c,   --format [yuv420]       set input format (yuv420, yuv420_10, yuv420_10_packed)
  -r,   --framerate [60]        temporal rate (framerate numerator) e.g. 25,30, 30000, 50,60, 60000 
        --framescale [1]        temporal scale (framerate denominator) e.g. 1, 1001 
        --fps [60/1]            Framerate as int or fraction (num/denom) 

### VP8, VP9, AV1

Open Media Alliance created VP8, VP9, VP1 to counteract the H.264|MPEG-4 AVC, H.265|HEVC potential licensing woes. H.266/VVC seems to be encumbered by licensing issues as well. VP8 was deprecated, superceded by VP9, which itself was superceded by AV1.


In [79]:
!ffmpeg -h encoder=vp8

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /

In [100]:
def compress_video(video_full_path, output_file_name, target_size):
    # Reference: https://en.wikipedia.org/wiki/Bit_rate#Encoding_bit_rate
    min_audio_bitrate = 32000
    max_audio_bitrate = 256000

    # get AV info
    probe = ffmpeg.probe(video_full_path)
    duration = float(probe["format"]["duration"])

    audio_bitrate = 0
    
    # if there's audio
    if ffmpeg.probe(video_full_path, select_streams="a")["streams"]:
        # Audio bitrate, in bps.
        audio_bitrate = float(
            next((s for s in probe["streams"] if s["codec_type"] == "audio"), None)["bit_rate"]
        )
    
    # Target total bitrate, in bps.
    target_total_bitrate = (target_size * 1024 * 8) / (1.073741824 * duration)

    if audio_bitrate > 0:
        # Target audio bitrate, in bps
        if 10 * audio_bitrate > target_total_bitrate:
            audio_bitrate = target_total_bitrate / 10

            if audio_bitrate < min_audio_bitrate < target_total_bitrate:
                audio_bitrate = min_audio_bitrate

            elif audio_bitrate > max_audio_bitrate:
                audio_bitrate = max_audio_bitrate
            
    # Target video bitrate, in bps.
    video_bitrate = target_total_bitrate - audio_bitrate

    i = ffmpeg.input(video_full_path)
    
    # in this case, x264, 2 passes, MP4 container, using libx264
    # https://ffmpeg.org/ffmpeg.html#Main-options
    # https://ffmpeg.org/ffmpeg.html#toc-Video-Options
    #
    # -b:v = video bitrate, -b:a = audio bitrate, AAC audio container pass 2
    # -c[:stream] codec, i.e, -c:v x264 for video stream codec x264
    # same as -vcodec codec, -codec:v
    # -f = format = container mp4, -y overwrite file
    # for audio stream -c:a copy = just copy the audio output
    # 
    # https://ffmpeg.org/ffmpeg.html#Advanced-Video-options
    # -pix_fmt (use -pix_fmts to display list):
    # yuv420p | yuv422p | yuv410p | yuv411p | yuv444p
    # for 10bit, p10, 9bit, p9, 16bit, p16, i.e, yuv444p10be|le
    # xyz12le|be. yuv440p10le|be = 20bit, yuv440p12le|be = 24bit
    #
    # Stats: -psnr = calculate psnr of compressed frames, -vstats_file <file> (stats)
    # -qphist (show QP histogram). -dc precision (intra dc precision)
    # -init_hw_device cuda:1, second dev on the system or
    # -init_hw_device cuda:0,primary_ctx=1
    #
    # stats:
    # https://ffmpeg.org/ffmpeg-filters.html#codecview
    # motion vectors view, etc, ffplay -flags2 +export_mvs input.mp4 -vf codecview=mv=pf+bf+bb
    #
    # https://ffmpeg.org/ffmpeg-codecs.html
    # https://ffmpeg.org/ffmpeg-codecs.html#codec_002doptions
    # flags <flags> : mv4 | qpel , g <int> (GOP size, 12) | qcomp <float> video quantizer scale
    # scale compression VBR, constant in rate control equation, [0, 1]
    # bf <int> number of B-frames between non-B-frames, [0-16], 0=disabled, -1=auto from codec
    # b_qfactgor = qp factor between P and B frames
    # DCT = auto | fastint | int | mmx | faan (AAN DCT floating point)
    # EC concealment strategy, guess_mvs | deblock | favor_inter<code="language-python">%%latex
    # mbd <int> macroblock choice: simple (def) | bits (fewest bits) | rd (best rate distortion)
    # export_side_data <flags>
    # venc_params
    # 
    # dc <int> (intra_dc_precision)
    # profile <int> (codec profile, varies w codec) | level <int> (codec profile)
    # bidir_refine <int> refine motion vectors used in-between macroblocks
    #
    # https://trac.ffmpeg.org/wiki/Encode/H.264
    # constant rate factor CRF in [0,51], 0=lossless, 23 = default, 51 = lossy, 17/18 perceptually lossless
    # preset: (ultra|super|very)fast, medium, slow, slower, veryslow, placebo
    # tune: film | animation | grain | stillimage | fastdecode | zerolatency (| psnr | ssim = stats)
    # ffmpeg -i input -c:v libx264 -preset slow -crf 22 -x264-params keyint=123:min-keyint=20 -c:a copy output.mkv
    # see ffmpeg -h encoder=libx264
    #

    ffmpeg.output(i, os.devnull,
                  **{
                      "c:v": "libx264",
                      "b:v": video_bitrate,
                      "pass": 1,
                      "f": "mp4",
                      "preset" : "slow",
                      "tune" : "animation",
                      "x264-params" : "dct=faan",
                      "export_side_data" : "venc_params",
                  }
                  ).overwrite_output().run()
    
    # 2nd pass
    ffmpeg.output(i, output_file_name,
                  **{
                      "c:v": "libx264",
                      "b:v": video_bitrate,
                      "pass": 2,
                      "preset" : "slow",
                      "tune" : "animation",
                      "x264-params" : "dct=faan",
                      "export_side_data" : "venc_params",
                      "c:a": "copy"
                  }
                  ).overwrite_output().run()
    

In [101]:
# compress video to MB size
target_size = 50 * 1024

compress_video(config["VIDEO_IN"][0], config["VIDEO_OUT"][0], target_size)

ffmpeg version n4.4.1-2-gcc33e73618 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Ubuntu 10.3.0-1ubuntu1~20.04)
  configuration: --prefix=/opt/local --enable-gpl --enable-version3 --enable-nonfree --enable-shared --enable-opencl --enable-opengl --enable-vulkan --enable-libxvid --enable-libx265 --enable-libx264 --enable-libwebp --enable-libvpx --enable-libvorbis --enable-libtheora --enable-libsrt --enable-libspeex --enable-libsoxr --enable-librtmp --enable-librsvg --enable-libopus --enable-libopenmpt --enable-libcaca --cpu=znver2 --enable-pic --enable-lto --enable-avx2 --enable-libkvazaar --enable-libvmaf --enable-libdav1d --enable-libaom --x86asmexe=yasm
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 /